# Проект: отток клиентов банка

В данном проекте мы имеем данные, содержащую информацию о клиенте и о том, ушёл ли клиент из банка. Наша цель построить модель, которая бы предсказывала уход клиента из банка. Модель должна иметь показатель F1-меры не менее 0,59. Также будем следить за метрикой AUC-ROC. Решать предстоит задачу бинарной классификации.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Баланс-классов" data-toc-modified-id="Баланс-классов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Баланс классов</a></span><ul class="toc-item"><li><span><a href="#Балансировка-с-помощью-class_weight" data-toc-modified-id="Балансировка-с-помощью-class_weight-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Балансировка с помощью class_weight</a></span></li><li><span><a href="#Upsampling" data-toc-modified-id="Upsampling-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Upsampling</a></span></li><li><span><a href="#Downsampling" data-toc-modified-id="Downsampling-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Downsampling</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Итоговой-тестирование-и-проверка-на-адекватность" data-toc-modified-id="Итоговой-тестирование-и-проверка-на-адекватность-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Итоговой тестирование и проверка на адекватность</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Подготовка данных

Для начала прочитаем данные и посмотрим на информацию о них.

In [24]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle

In [25]:
data = pd.read_csv('./Churn.csv')
display(data.head())
data.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Видим что в столбце `Tenure` имеются пропущенные значения. Этот столбец показывает сколько лет человек является клиентом банка. Для дальнейшей работы и обучения моделей нам необходимо убрать пропуски. Заполним отсутствующие значения медианой. Также преобразуем данный столбец к формату int, так как года представлены целыми числами.

In [26]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median()).astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


В нашем исследовании признаками будут являться столбцы с 3 по 12 (от `CreditScore` до `EstimatedSalary`). Первые 3 стобца - индекс строки, идентификатор клиента и фамилию не берём, так как они являются просто идентификаторами, которые помогают отличать одного клиента от другого.

In [27]:
features = data.loc[:, 'CreditScore':'EstimatedSalary']
target = data['Exited']

Среди признаков есть 2 столбца, содержащие строковые значения - `Geography` и `Gender`. Для работы с моделями машинного обучения нам необходимо преобразовать эти значения в числа. Для этого воспользуемся техникой прямого кодирования.

In [28]:
features = pd.get_dummies(features, columns=['Geography', 'Gender'], drop_first=True)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  int64  
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Geography_Germany  10000 non-null  uint8  
 9   Geography_Spain    10000 non-null  uint8  
 10  Gender_Male        10000 non-null  uint8  
dtypes: float64(2), int64(6), uint8(3)
memory usage: 654.4 KB


Разобъём данные на 3 выборки - обучающую 60%, валидационную 20% (на ней будем проводить настройку моделей) и тестовую 20% (на ней будем проводить финальное тестирование).

In [29]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(features, target, 
                                                                test_size=0.4, random_state=12345, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid_test, target_valid_test, 
                                                                test_size=0.5, random_state=12345, stratify=target_valid_test)

In [30]:
print('Обучающая выборка:\n', target_train.value_counts(), '\n')
print('Валидационная выбока:\n', target_valid.value_counts(), '\n')
print('Тестовая выборка:\n', target_test.value_counts(), '\n')

Обучающая выборка:
 0    4778
1    1222
Name: Exited, dtype: int64 

Валидационная выбока:
 0    1592
1     408
Name: Exited, dtype: int64 

Тестовая выборка:
 0    1593
1     407
Name: Exited, dtype: int64 



In [31]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(6000, 11)
(2000, 11)
(2000, 11)


Признаки в нашем наборе данных имеют разный масштаб. Это может привести к неправильному обучению модели. Чтобы этого избежать применим метод стандартизации данных. Стандартизировать требуется только числовые признаки.

In [32]:
pd.options.mode.chained_assignment = None

numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Исследование задачи

Нам необходимо определить, есть ли в обучающей выборке дисбаланс классов. Дисбаланс может негативно сказаться на обучении модели. Посчитаем, сколько в целевом признаке встречается класс "1", а сколько "0".

In [33]:
target_train_ones = target_train[target_train == 1]
target_train_zeros = target_train[target_train == 0]
print('Доля класса "1":', len(target_train_ones)/(len(target_train_ones)+len(target_train_zeros)))
print('Доля класса "0":', len(target_train_zeros)/(len(target_train_ones)+len(target_train_zeros)))

Доля класса "1": 0.20366666666666666
Доля класса "0": 0.7963333333333333


Нулевой класс встречается в нашей выборке в 4 раза чаще. Это говорит нам о дисбалансе. Но для начала обучим модели на несбалансированной выборке, чтобы иметь возможность сравнить, как дисбаланс влияет на качество моделей и метрики.

Обучим решающее дерево и попробуем найти глубину, которая даст наибольшее значение для F1.

In [34]:
tree_best_depth = 0
best_split = 0
tree_f1_best = 0
model_tree = None
for depth in range(1, 16):
    for split in range(2, 21):
        model = DecisionTreeClassifier(max_depth=depth, min_samples_split=split, random_state=12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        tree_f1 = f1_score(target_valid, predictions)
        if tree_f1 > tree_f1_best:
            tree_f1_best = tree_f1
            tree_best_depth = depth
            best_split = split
            model_tree = model
        
print('Глубина решающего дерева:', tree_best_depth)
print('Минимальное количество выборок для разделения узла:', best_split)
print('F1-мера для решающего дерева при дисбалансе:', tree_f1_best)
probabilities = model_tree.predict_proba(features_valid)
print('AUC-ROC для решающего дерева при дисбалансе:', roc_auc_score(target_valid, probabilities[:, 1]))

Глубина решающего дерева: 7
Минимальное количество выборок для разделения узла: 5
F1-мера для решающего дерева при дисбалансе: 0.5977011494252873
AUC-ROC для решающего дерева при дисбалансе: 0.8297315314316682


Теперь рассмотрим модель случайного леса и попробуем подобрать наилучшие значения для глубины дерева и количества деревьев.

In [35]:
forest_best_depth = 0
best_n_estimators = 0
forest_f1_best = 0
model_forest = None

for number in range(10, 101, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(max_depth=depth, n_estimators=number, random_state=12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        forest_f1 = f1_score(target_valid, predictions)
        if forest_f1 > forest_f1_best:
            forest_best_depth = depth
            best_n_estimators = number
            forest_f1_best = forest_f1
            model_forest = model

print('Максимальная глубина леса:', forest_best_depth)
print('Наилучшее количество деревьев леса:', best_n_estimators)
print('F1-мера для случайного леса при дисбалансе:', forest_f1_best)

Максимальная глубина леса: 15
Наилучшее количество деревьев леса: 60
F1-мера для случайного леса при дисбалансе: 0.6283185840707965


In [36]:
probabilities = model_forest.predict_proba(features_valid)
print('AUC-ROC для случайного леса при дисбалансе:', roc_auc_score(target_valid, probabilities[:, 1]))

AUC-ROC для случайного леса при дисбалансе: 0.8644540102473149


Наилучшие результаты по обеим метрикам показывает модель случайного леса.

## Баланс классов

### Балансировка с помощью class_weight

Теперь пришло время обучить модели на сбалансированных данных. Есть несколько способов борьбы с дисбалансом классов. Можно увеличить частоту редкого класса, размножив его объекты. Или же наоборот, удалить часть объектов частого класса. Мы воспользуемся самым простым способом и при обучении моделей укажем в качестве параметра class_weight='balanced'. В таком случае модель назначит более редкому классу более высокий вес.

In [37]:
balanced_depth = 0
balanced_split = 0
f1_tree_balanced = 0
model_tree_balanced = None

for depth in range(1, 16):
    for split in range(2, 21):
        model = DecisionTreeClassifier(max_depth=depth, min_samples_split=split, class_weight='balanced', random_state=12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        tree_f1 = f1_score(target_valid, predictions)
        if tree_f1 > f1_tree_balanced:
            f1_tree_balanced = tree_f1
            balanced_split = split
            balanced_depth = depth
            model_tree_balanced = model
        
print('Глубина решающего дерева:', balanced_depth)
print('Минимальное количество выборок для разделения узла:', balanced_split)
print('F1-мера для решающего дерева:', f1_tree_balanced)
probabilities = model_tree_balanced.predict_proba(features_valid)
print('AUC-ROC для решающего дерева:', roc_auc_score(target_valid, probabilities[:, 1]))

Глубина решающего дерева: 6
Минимальное количество выборок для разделения узла: 4
F1-мера для решающего дерева: 0.5780240073868882
AUC-ROC для решающего дерева: 0.8272613065326633


А вот решающее дерево показало результаты хуже, чем при дисбалансе классов.

In [38]:
forest_best_depth = 0
forest_f1_best = 0
best_n_estimators = 0
model_forest_balanced = None

for number in range(10, 101, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(max_depth=depth, n_estimators=number, 
                                   class_weight='balanced', random_state=12345)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        forest_f1 = f1_score(target_valid, predictions)
        if forest_f1 > forest_f1_best:
            forest_best_depth = depth
            best_n_estimators = number
            forest_f1_best = forest_f1
            model_forest_balanced = model

In [39]:
print('Наилучшая максимальная глубина леса:', forest_best_depth)
print('Наилучшее количество деревьев:', best_n_estimators)
print('F1-мера для случайного леса:', forest_f1_best)
probabilities = model_forest_balanced.predict_proba(features_valid)
print('AUC-ROC случайного леса:', roc_auc_score(target_valid, probabilities[:, 1]))

Наилучшая максимальная глубина леса: 10
Наилучшее количество деревьев: 30
F1-мера для случайного леса: 0.6499402628434886
AUC-ROC случайного леса: 0.8697808897428319


### Upsampling

Метод upsampling подразумевает увеличение частоты редкого класса. Для этого разобъём обучающую выборку на классы, скопируем несколько раз редкий класс и перемешаем данные. Учитывая, что нулевой класс у нас встречается примерно в 4 раза реже, то при копировании умножать будем на 4. После проверим, как работают модели на данных после upsampling.

In [40]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4)
print(target_train_upsampled.value_counts())

1    4888
0    4778
Name: Exited, dtype: int64


In [41]:
tree_best_depth = 0
best_split = 0
tree_f1_best = 0
model_tree_upample = None
for depth in range(1, 16):
    for split in range(2, 21):
        model = DecisionTreeClassifier(max_depth=depth, min_samples_split=split, random_state=12345)
        model.fit(features_train_upsampled, target_train_upsampled)
        predictions = model.predict(features_valid)
        tree_f1 = f1_score(target_valid, predictions)
        if tree_f1 > tree_f1_best:
            tree_f1_best = tree_f1
            best_split = split
            tree_best_depth = depth
            model_tree_upample = model
        
print('Глубина решающего дерева, метод upsampling:', tree_best_depth)
print('Минимальное количество выборок для разделения узла:', best_split)
print('F1-мера для решающего дерева, метод upsampling:', tree_f1_best)
probabilities = model_tree_upample.predict_proba(features_valid)
print('AUC-ROC для решающего дерева, метод upsampling:', roc_auc_score(target_valid, probabilities[:, 1]))

Глубина решающего дерева, метод upsampling: 6
Минимальное количество выборок для разделения узла: 10
F1-мера для решающего дерева, метод upsampling: 0.5780240073868882
AUC-ROC для решающего дерева, метод upsampling: 0.827699311508523


In [42]:
forest_best_depth = 0
best_n_estimators = 0
forest_f1_best = 0
model_forest_upsample = None

for number in range(10, 101, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(max_depth=depth, n_estimators=number, random_state=12345)
        model.fit(features_train_upsampled, target_train_upsampled)
        predictions = model.predict(features_valid)
        forest_f1 = f1_score(target_valid, predictions)
        if forest_f1 > forest_f1_best:
            forest_best_depth = depth
            best_n_estimators = number
            forest_f1_best = forest_f1
            model_forest_upsample = model

print('Максимальная глубина леса, метод upsampling:', forest_best_depth)
print('Наилучшее количество деревьев леса, метод upsampling:', best_n_estimators)
print('F1-мера для случайного леса, метод upsampling:', forest_f1_best)
probabilities = model_forest_upsample.predict_proba(features_valid)
print('AUC-ROC для случайного, метод upsampling:', roc_auc_score(target_valid, probabilities[:, 1]))

Максимальная глубина леса, метод upsampling: 10
Наилучшее количество деревьев леса, метод upsampling: 30
F1-мера для случайного леса, метод upsampling: 0.6511134676564156
AUC-ROC для случайного, метод upsampling: 0.8664554389595034


### Downsampling

Данный метод подразумевает уменьшение большого класса и, как следствие, всей выборки в целом. Сначала выборка разбивается на классы, случайным образом отбрасываются объекты большего класса. Затем выборка снова собирается и перемешивается.

In [43]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_train_downsampled, target_train_downsampled = downsample(features_train, target_train, 0.25)
print(target_train_downsampled.value_counts())

1    1222
0    1194
Name: Exited, dtype: int64


In [44]:
tree_best_depth = 0
best_split = 0
tree_f1_best = 0
model_tree_downsample = None
for depth in range(1, 16):
    for split in range(2, 21):
        model = DecisionTreeClassifier(max_depth=depth, min_samples_split=split, random_state=12345)
        model.fit(features_train_downsampled, target_train_downsampled)
        predictions = model.predict(features_valid)
        tree_f1 = f1_score(target_valid, predictions)
        if tree_f1 > tree_f1_best:
            tree_f1_best = tree_f1
            best_split = split
            tree_best_depth = depth
            model_tree_downsample = model
        
print('Глубина решающего дерева, метод downsampling:', tree_best_depth)
print('Минимальное количество выборок для разделения узла:', best_split)
print('F1-мера для решающего дерева, метод downsampling:', tree_f1_best)
probabilities = model_tree_downsample.predict_proba(features_valid)
print('AUC-ROC для решающего дерева, метод downsampling:', roc_auc_score(target_valid, probabilities[:, 1]))

Глубина решающего дерева, метод downsampling: 6
Минимальное количество выборок для разделения узла: 7
F1-мера для решающего дерева, метод downsampling: 0.5971563981042654
AUC-ROC для решающего дерева, метод downsampling: 0.8525763006207507


In [45]:
forest_best_depth = 0
best_n_estimators = 0
forest_f1_best = 0
model_forest_downsample = None

for number in range(10, 101, 10):
    for depth in range (1, 16):
        model = RandomForestClassifier(max_depth=depth, n_estimators=number, random_state=12345)
        model.fit(features_train_downsampled, target_train_downsampled)
        predictions = model.predict(features_valid)
        forest_f1 = f1_score(target_valid, predictions)
        if forest_f1 > forest_f1_best:
            forest_best_depth = depth
            best_n_estimators = number
            forest_f1_best = forest_f1
            model_forest_downsample = model

print('Максимальная глубина леса, метод upsampling:', forest_best_depth)
print('Наилучшее количество деревьев леса, метод upsampling:', best_n_estimators)
print('F1-мера для случайного леса, метод upsampling:', forest_f1_best)
probabilities = model_forest_downsample.predict_proba(features_valid)
print('AUC-ROC для случайного, метод upsampling:', roc_auc_score(target_valid, probabilities[:, 1]))

Максимальная глубина леса, метод upsampling: 7
Наилучшее количество деревьев леса, метод upsampling: 30
F1-мера для случайного леса, метод upsampling: 0.6375121477162294
AUC-ROC для случайного, метод upsampling: 0.8654593432850527


### Вывод

Мы рассмотрели разные методы борьбы с дисбалансом. Все модели показали лучшие результаты по сравнению с моделями, обученными на несбалансированных классах. При этом разница метрики AUC-ROC невелика. Разница в методах есть и отличается для каждой модели. Например модель решающего дерева показала лучшие результаты при использовании метода downsampling, а модель случайного леса - при upsampling. Для тестирования выберем лучшую модель. Ею является модель случайного леса с глубиной 10, количеством деревьев 30, обученная на выборке после upsampling.

## Итоговой тестирование и проверка на адекватность

Протестируем нашей наиболее успешной модели на тестовой выборке.

In [46]:
predictions = model_forest_upsample.predict(features_test)
print('F1-мера случайного леса на тестовой выборке:', f1_score(target_test, predictions))
probabilities = model_forest_upsample.predict_proba(features_test)
print('AUC-ROC случайного леса на тестовой выборке:', roc_auc_score(target_test, probabilities[:, 1]))

F1-мера случайного леса на тестовой выборке: 0.6013071895424837
AUC-ROC случайного леса на тестовой выборке: 0.8489938320446795


Наша модель преодолела порог в 0,59 для F1-меры. Также как видно из AUC-ROC модель предсказывает лучше случайной модели (AUC-ROC случайной модели = 0,5).

## Вывод

В данном проекте мы выбирали модель, которая бы предсказывала уход клиентов из банка. Мы рассмотрели несколько моделей (дерево решений, случайный лес) и их гиперпараметры. Для оценки использовали метрику F1-мера. Наилучшие результаты показала модель случайного леса.
Также мы исследовали влияние дисбаланса классов на модель. Дисбаланс классов снижает качество обучения и как следствие предсказательную способность модели. Метрики F1 и AUC-ROC у моделей, обученных на несбалансированных данных были ниже. Хотя на AUC-ROC дисбаланс повлиял не так сильно, как на F1-меру.